#보스턴 주택 가격 예측
- 보스턴 주택 가격 데이터셋은 연속적인 값을 예측하는 회귀 문제
- 1970년대 보스턴 지역의 범죄율, 토지 지역의 비율, 방의 개수 등 정답을 포함한 총 14개 특성으로 이루어짐

데이터 다운

In [11]:
from tensorflow.keras.datasets.boston_housing import load_data
(x_train, y_train),(x_test, y_test) = load_data(path='boston_housing.npz', test_split=0.2, seed=777)

In [12]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(404, 13) (404,)
(102, 13) (102,)


In [13]:
print(x_train)
print(y_train)

[[2.51990e-01 0.00000e+00 1.05900e+01 ... 1.86000e+01 3.89430e+02
  1.80600e+01]
 [1.58603e+01 0.00000e+00 1.81000e+01 ... 2.02000e+01 7.68000e+00
  2.43900e+01]
 [7.40389e+00 0.00000e+00 1.81000e+01 ... 2.02000e+01 3.14640e+02
  2.64000e+01]
 ...
 [9.88430e-01 0.00000e+00 8.14000e+00 ... 2.10000e+01 3.94540e+02
  1.98800e+01]
 [8.01400e-02 0.00000e+00 5.96000e+00 ... 1.92000e+01 3.96900e+02
  8.77000e+00]
 [3.69200e-01 0.00000e+00 9.90000e+00 ... 1.84000e+01 3.95690e+02
  9.28000e+00]]
[22.5  8.3 17.2 25.  28.5 23.  18.9 50.  15.6 38.7 24.6 23.3  9.5 20.
 18.3 36.4  5.  19.4 28.1 17.8 19.6 24.5 10.4 23.7 19.4 23.9 21.4 29.9
 24.4 23.1 25.  30.1 22.8 22.8 22.  24.8 15.6 22.  19.6 13.1 22.5 18.9
 14.6 13.3 17.4 20.8 10.8 21.4 17.2 13.8 28.7 35.1 10.2 21.7 20.5 25.
 21.1 15.6 23.6 23.2 28.7 16.6 26.2  9.6 18.2 31.7 42.8 16.1 22.6 18.1
 23.8 11.8 16.  21.5 13.9 11.7 44.8 20.1 50.  22.1 20.6 14.6 48.8 22.9
 14.4 23.9 15.6 19.9 20.5  8.7 26.5 20.3 23.8 44.  18.5 17.5 22.8 14.3
 10.9 36.1 25

In [14]:
# 총데이터는 506개로 상당히 적은 편에 속함
# 학습데이터 404개, 테스트데이터 102개
# 데이터는 13개 특성이 있으며, 레이블은 주택 가격의 중간 가격을 나타냄
# 특성의 스케일이 다르기 따문에 표준화 필요

데이터 전처리 및 검증 데이터 셋 만들기

In [15]:
import numpy as np
mean = np.mean(x_train, axis=0)
std = np.std(x_train, axis=0)

x_train = (x_train-mean)/std
x_test = (x_test-mean)/std

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.33, random_state=777)

모델구성 및 학습

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(13,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [18]:
# 데이터 갯수가 적을 수록, 모델이 깊어지면 과대적합 확률이 높음
# 마지막 Dense층에서 softmax함수를 쓰지 않은 이유는, 보스턴 주택 가격은 점답 범위가 제한되지 안힉 떄문 입니다

In [20]:
history = model.fit(x_train, y_train, epochs=300, validation_data=(x_val, y_val))
model.evaluate(x_test, y_test)

Epoch 1/300
9/9 [==============================] - 0s 6ms/step - loss: 480.3722 - mae: 20.2531 - val_loss: 518.6987 - val_mae: 20.6101
Epoch 2/300
9/9 [==============================] - 0s 9ms/step - loss: 444.0579 - mae: 19.4035 - val_loss: 474.1823 - val_mae: 19.6255
Epoch 3/300
9/9 [==============================] - 0s 8ms/step - loss: 400.3170 - mae: 18.3438 - val_loss: 421.3192 - val_mae: 18.3840
Epoch 4/300
9/9 [==============================] - 0s 7ms/step - loss: 348.7265 - mae: 17.0106 - val_loss: 361.3052 - val_mae: 16.8541
Epoch 5/300
9/9 [==============================] - 0s 6ms/step - loss: 291.4114 - mae: 15.4016 - val_loss: 296.0229 - val_mae: 14.9869
Epoch 6/300
9/9 [==============================] - 0s 7ms/step - loss: 232.1050 - mae: 13.5169 - val_loss: 230.6070 - val_mae: 12.7958
Epoch 7/300
9/9 [==============================] - 0s 6ms/step - loss: 172.7830 - mae: 11.3688 - val_loss: 173.7135 - val_mae: 10.5762
Epoch 8/300
9/9 [==============================] - 0s 7

[9.659204483032227, 2.2221596240997314]

In [21]:
# 실제값과 예측값 사이에 평균 2200달러 정도의 차이가 있음

**K-폴드 교차검증** : 데이터셋이 적은 경우 모델의 성능 향상에 좋은 방법

In [27]:
from tensorflow.keras.datasets.boston_housing import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
from sklearn.model_selection import KFold

(x_train, y_train),(x_test, y_test) = load_data(path='boston_housing.npz', test_split=0.2, seed=777)

mean = np.mean(x_train, axis=0)
std = np.std(x_train, axis=0)

x_train = (x_train-mean)/std
x_test = (x_test-mean)/std

# KFold
k=3
kfold = KFold(n_splits=k, random_state=777, shuffle=True)

def get_model():
  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape=(13,)))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer='adam', loss='mse', metrics=['mae'])
  return model

mae_list = []

for train_idx, val_idx in kfold.split(x_train):
  x_train_fold, x_val_fold = x_train[train_idx], x_train[val_idx]
  y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

  model = get_model()

  model.fit(x_train_fold, y_train_fold, epochs=300, validation_data=(x_val_fold, y_val_fold))

  _, test_mae = model.evaluate(x_test, y_test)
  mae_list.append(test_mae)

Epoch 1/300
9/9 [==============================] - 1s 26ms/step - loss: 567.4926 - mae: 22.1434 - val_loss: 618.0383 - val_mae: 22.6467
Epoch 2/300
9/9 [==============================] - 0s 6ms/step - loss: 540.8624 - mae: 21.5574 - val_loss: 589.6824 - val_mae: 22.0362
Epoch 3/300
9/9 [==============================] - 0s 8ms/step - loss: 513.8802 - mae: 20.9437 - val_loss: 559.6290 - val_mae: 21.3641
Epoch 4/300
9/9 [==============================] - 0s 8ms/step - loss: 484.4637 - mae: 20.2465 - val_loss: 524.5849 - val_mae: 20.5354
Epoch 5/300
9/9 [==============================] - 0s 7ms/step - loss: 448.4068 - mae: 19.3551 - val_loss: 480.8390 - val_mae: 19.4638
Epoch 6/300
9/9 [==============================] - 0s 9ms/step - loss: 404.2092 - mae: 18.2123 - val_loss: 428.1881 - val_mae: 18.0955
Epoch 7/300
9/9 [==============================] - 0s 9ms/step - loss: 352.3748 - mae: 16.8165 - val_loss: 366.7943 - val_mae: 16.4087
Epoch 8/300
9/9 [==============================] - 0s 

In [28]:
print(mae_list)
print(np.mean(mae_list))

[2.231443166732788, 2.186310291290283, 2.263354778289795]
2.2270360787709556
